In [4]:
import os

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [5]:
descriptions = pd.read_csv(os.path.join('data', 'descriptions.csv')).rename(columns={'movie': 'item'})

In [6]:
descriptions.head()

,id,description
0,0,"Captain Barbossa, long believed to be dead, ha..."
1,1,The seemingly invincible Spider-Man goes up ag...
2,2,Superman returns to discover his 5-year absenc...
3,3,Quantum of Solace continues the adventures of ...
4,4,Captain Jack Sparrow works his way out of a bl...


In [ ]:
texts = descriptions.description
texts = texts.apply(lambda x: x.strip().split())
dictionary = Dictionary(texts.values)
dictionary.filter_extremes(keep_n=8000)
eos_id = len(dictionary.keys())

# to index list
texts = texts.apply(lambda x: dictionary.doc2idx(x, unknown_word_index=eos_id))
texts = texts.apply(lambda x: np.array([a for a in x if a != eos_id]))
max_sentence_length = max(texts.apply(len)) if max_sentence_length is None else min(max(texts.apply(len)), max_sentence_length)

# padding
texts = texts.apply(lambda x: x[:max_sentence_length])
texts = texts.apply(lambda x: np.pad(x, (0, max_sentence_length - len(x)), 'constant', constant_values=(0, eos_id)))

# change types
texts = texts.apply(lambda x: x.astype(np.int32))
descriptions.id = descriptions.id.astype(np.int32)

return np.array(list(descriptions.id.values)), np.array(list(texts.values)), len(dictionary.keys()) + 1
